In [148]:
import numpy as np
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.SubsMat import MatrixInfo as matlist

In [149]:
def nw(s1, s2, M, d):
    def score_matrix(M, c1, c2):
        if (c1,c2) in M.keys():
            return M[(c1,c2)]
        else:
            return M[(c2,c1)]
    nx = len(s1)
    ny = len(s2)
    D = np.zeros((nx+1, ny+1))
    for i in range(nx+1): D[i,0] = (-d)*i
    for j in range(ny+1): D[0,j] = (-d)*j
    for i in range(1,nx+1):
        for j in range(1,ny+1):
            top = D[i, j-1]
            left = D[i-1, j]
            diag = D[i-1, j-1]
            val = score_matrix(M, s1[i-1], s2[j-1])
            D[i,j] = max(top-d, left-d, diag + val)
    alignA = ""
    alignB = ""
    i=nx
    j=ny
    score = 0
    while (i>0 or j>0):
        val = score_matrix(M, s1[i-1], s2[j-1])
        if (i>0 and j>0 and D[i,j] == D[i-1, j-1] + val):
            score += val
            alignA = s1[i-1] + alignA
            alignB = s2[j-1] + alignB
            i -= 1
            j -= 1
        elif (i >0 and D[i,j] == D[i-1, j] - d):
            score -= d
            alignA = s1[i-1] + alignA
            alignB = "-" + alignB
            i -= 1
        else:
            score -= d
            alignB = s2[j-1] + alignB
            alignA = "-" + alignA
            j -= 1
    return (alignA, alignB), score

In [150]:
nw('KELAS', 'ELS', matlist.blosum62, 1)

(('KELAS', '-EL-S'), 11)

In [151]:
for a in pairwise2.align.globalds("KELAS", "ELS", matlist.blosum62, -1, -1):
    print(format_alignment(*a))
    print(a[2])

KELAS
 || |
-EL-S
  Score=11

11.0


In [152]:
import pandas as pd

In [155]:
df = pd.read_csv("cycsamples_en.csv")
cytochrome_c = df['sequence']

nrow = len(df)
type(cytochrome_c[0])

str

In [159]:
result = np.zeros((nrow, nrow))

for i in range(nrow):
    for j in range(nrow):
        result[i,j] = nw(cytochrome_c[i], cytochrome_c[j], matlist.blosum62, 1)[1]

result_data = pd.DataFrame(result, index=df['name'], columns=df['name'])
result_data.to_csv('alignment.csv', encoding="utf_8_sig")